In [ ]:
import math
import logging
from functools import partial
from collections import OrderedDict
from einops import rearrange, repeat

import torch
import torch.nn as nn
import torch.nn.functional as F

from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.helpers import load_pretrained
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model


import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads

        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

class GestureClassification(nn.Module):
    def __init__(self, num_frame=101, num_joints=21, in_chans=3, embed_dim_ratio=32, depth=4,
                 num_heads=8, mlp_ratio=2., qkv_bias=True, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.2,  norm_layer=None, num_classes = 0):
        """    ##########hybrid_backbone=None, representation_size=None,
        Args:
            num_frame (int, tuple): input frame number
            num_joints (int, tuple): joints number
            in_chans (int): number of input channels, 2D joints have 2 channels: (x,y)
            embed_dim_ratio (int): embedding dimension ratio
            depth (int): depth of transformer
            num_heads (int): number of attention heads
            mlp_ratio (int): ratio of mlp hidden dim to embedding dim
            qkv_bias (bool): enable bias for qkv if True
            qk_scale (float): override default qk scale of head_dim ** -0.5 if set
            drop_rate (float): dropout rate
            attn_drop_rate (float): attention dropout rate
            drop_path_rate (float): stochastic depth rate
            norm_layer: (nn.Module): normalization layer
        """
        super().__init__()

        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        embed_dim = embed_dim_ratio * num_joints   #### temporal embed_dim is num_joints * spatial embedding dim ratio
        out_dim = num_classes

        ### spatial patch embedding
        self.Spatial_patch_to_embedding = nn.Linear(in_chans, embed_dim_ratio)
        self.Spatial_pos_embed = nn.Parameter(torch.zeros(1, num_joints, embed_dim_ratio))

        self.Temporal_pos_embed = nn.Parameter(torch.zeros(1, num_frame, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)


        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule

        self.Spatial_blocks = nn.ModuleList([
            Block(
                dim=embed_dim_ratio, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])

        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])

        self.Spatial_norm = norm_layer(embed_dim_ratio)
        self.Temporal_norm = norm_layer(embed_dim)

        ####### A easy way to implement weighted mean
        self.weighted_mean = torch.nn.Conv1d(in_channels=num_frame, out_channels=1, kernel_size=1)

        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim , out_dim),
        )


    def Spatial_forward_features(self, x):
        b, _, f, p = x.shape  ##### b is batch size, f is number of frames, p is number of joints
        x = rearrange(x, 'b c f p  -> (b f) p  c', )


        x = self.Spatial_patch_to_embedding(x)
        x += self.Spatial_pos_embed
        x = self.pos_drop(x)

        for blk in self.Spatial_blocks:
            x = blk(x)

        x = self.Spatial_norm(x)
        x = rearrange(x, '(b f) w c -> b f (w c)', f=f)
        return x

    def forward_features(self, x):
        b  = x.shape[0]
        x += self.Temporal_pos_embed
        x = self.pos_drop(x)
        for blk in self.blocks:
            x = blk(x)

        x = self.Temporal_norm(x)
        ##### x size [b, f, emb_dim], then take weighted mean on frame dimension, we only predict 3D pose of the center frame
        x = self.weighted_mean(x)
        x = x.view(b, 1, -1)
        return x


    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        b, _, _, p = x.shape
        x = self.Spatial_forward_features(x)
        x = self.forward_features(x)
        x = self.head(x)
        x = x.view(b, num_classes)

        return x


In [ ]:
class MyCustomDataset(Dataset):
    def __init__(self, root_dir, mode='train', sliding_window_size=101):
        self.root_dir = root_dir
        self.subjects = os.listdir(root_dir)
        self.frame_data = []  # Stores the information of each frame
        self.labels = [] # Stores the labels of each frame
        self.indexvalues = []
        self.sliding_window_size = sliding_window_size
        self._process_data(mode)

    def __len__(self):
        return len(self.frame_data)


    def __getitem__(self, index):
        center_frame = self.frame_data[index]
        label = self.labels[index]
        skeleton_number = center_frame['skeleton_number']

        skeleton_start_index = self.indexvalues[skeleton_number]['start_index']
        skeleton_end_index = self.indexvalues[skeleton_number]['end_index']

        start_index = index - self.sliding_window_size // 2
        end_index = index + self.sliding_window_size // 2

        if (start_index < skeleton_start_index):
          start_index = skeleton_start_index

        else:
          start_index = max(start_index, 0)

        if (end_index > skeleton_end_index):
          end_index = skeleton_end_index

        t = []

        for i in range(start_index, end_index + 1):
          frame_data = self.frame_data[i]
          joint_coordinates = frame_data['joint_coordinates']
          t.append(joint_coordinates)

        while len(t) < self.sliding_window_size:
          t.append([0.0] * 63)  # Assuming 21 joints with (x, y, z) coordinates

        t = torch.tensor(t, dtype=torch.float32)
        t = t.view(self.sliding_window_size, 21, 3)

        return t, label


    def _process_data(self, mode):
        frame_counter = 0  # Counter for sequential frame numbering
        skeleton_temp=0
        indices = []


        for subject in self.subjects:
            subject_dir = os.path.join(self.root_dir, subject)

            if mode == 'train':
                data_dir = os.path.join(subject_dir, 'TrainingData')
            elif mode == 'test':
                data_dir = os.path.join(subject_dir, 'TestingData')
            else:
                raise ValueError("Invalid mode. Mode should be 'train' or 'test'.")

            action_folders = os.listdir(subject_dir)

            for action_folder in action_folders:
                action_dir = os.path.join(subject_dir, action_folder)
                number_folders = os.listdir(action_dir)

                for number_folder in number_folders:
                    number_dir = os.path.join(action_dir, number_folder)
                    skeleton_file = os.path.join(number_dir, "skeleton.txt")

                    # Read and process the skeleton.txt file to extract frame-level data
                    frames, labels , start , end= self._read_skeleton_file(skeleton_file, frame_counter, action_folder , skeleton_temp)
                    indices.append({
                    'start_index': start,
                    'end_index': end,

                    })
                    self.frame_data.extend(frames)
                    self.labels.extend(labels)

                    # Update the frame counter based on the number of frames in the current skeleton file
                    frame_counter += len(frames)
                    skeleton_temp = skeleton_temp + 1

        self.indexvalues.extend(indices)



    def _read_skeleton_file(self, skeleton_file, frame_counter, action , skeleton_temp):
        frames = []
        labels = []
        start_ind = frame_counter
        temp_index = 0

        # Read and parse the skeleton.txt file to extract frame-level data
        with open(skeleton_file, 'r') as f:
            for line in f:
                # Parse the line and extract frame number, joint coordinates, and label
                frame_info = line.strip().split(' ')
                frame_number = frame_counter + int(frame_info[0])
                joint_coordinates = [float(coord) for coord in frame_info[1:]]
                label = action
                end_ind = frame_number

                # Append the frame data and label to the respective lists
                frames.append({
                    'frame_number': frame_number,
                    'joint_coordinates': joint_coordinates,
                    'skeleton_number' : skeleton_temp,

                })
                labels.append(label)
                temp_index = frame_number

        return frames, labels, start_ind, temp_index

In [ ]:
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Set the root directory of your dataset
train_root_dir = '/kaggle/input/fphadataset/FPHADataset/TrainingData'
test_root_dir = '/kaggle/input/fphadataset/FPHADataset/TestingData'

# Create an instance of the training dataset
train_dataset = MyCustomDataset(train_root_dir, mode='train')

# Create an instance of the testing dataset
test_dataset = MyCustomDataset(test_root_dir, mode='test')

# Set the batch size
batch_size = 32

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

train_dataset_length = len(train_dataset)

test_dataset_length = len(test_dataset)


# Set the number of output classes
num_classes = 45 # Number of classes in dataset

# Set the hyperparameters
#Sliding window size
num_frame = 101
num_joints = 21
in_chans = 3
embed_dim_ratio = 64
depth = 4
num_heads = 8
mlp_ratio = 2.
qkv_bias = True
qk_scale = None
drop_rate = 0.
attn_drop_rate = 0.
drop_path_rate = 0.2


# Create an instance of the PoseTransformer model
model = GestureClassification(num_frame=num_frame, num_joints=num_joints, in_chans=in_chans, embed_dim_ratio=embed_dim_ratio,
                        depth=depth, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                        drop_rate=drop_rate, attn_drop_rate=attn_drop_rate, drop_path_rate=drop_path_rate,
                        num_classes =num_classes)

# Move the model to the device
model = model.to(device)

# Set the optimizer and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Set the loss function
criterion = nn.CrossEntropyLoss()

# Set the number of training epochs
num_epochs = 10

# Create a label encoder
label_encoder = LabelEncoder()

# Fit label encoder on the training labels
label_encoder.fit(train_dataset.labels)

# Convert training labels to numerical values
train_labels_encoded = label_encoder.transform(train_dataset.labels)

# Convert the encoded labels to tensors
train_labels_tensor = torch.tensor(train_labels_encoded, dtype=torch.long).to(device)


# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    train_loss = 0.0
    train_correct = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)

        labels_encoded = label_encoder.transform(labels)  # Convert current batch labels to numerical values
        labels_tensor = torch.tensor(labels_encoded, dtype=torch.long).to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels_tensor)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Compute the training accuracy
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels_tensor).sum().item()

        # Update the training loss
        train_loss += loss.item() * inputs.size(0)

    # Compute the average training loss and accuracy
    train_loss = train_loss / len(train_dataset)
    train_accuracy = train_correct / len(train_dataset)

    # Print the training loss and accuracy for each epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    test_correct = 0
    test_predictions = []
    test_targets = []



    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)

            labels_encoded = label_encoder.transform(labels)  # Convert current batch labels to numerical values
            labels_tensor = torch.tensor(labels_encoded, dtype=torch.long).to(device)


            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels_tensor)

            # Compute the testing accuracy
            _, predicted = torch.max(outputs, 1)
            test_correct += (predicted == labels_tensor).sum().item()

            # Update the testing loss
            test_loss += loss.item() * inputs.size(0)

            # Collect predictions and targets for computing metrics
            test_predictions.extend(predicted.cpu().numpy())
            test_targets.extend(labels_encoded)

    # Compute the average testing loss and accuracy
    test_loss = test_loss / len(test_dataset)
    test_accuracy = test_correct / len(test_dataset)

    # Print the testing loss and accuracy for each epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


# Save the trained model
torch.save(model.state_dict(), '81_pose_transformer_model.pth')